# SS Scraper with filters

In [47]:
import requests
import copy
from bs4 import BeautifulSoup

headers_get = {
    'Referer': 'https://www.ss.lv/lv/transport/cars/',
    'Host': 'www.ss.lv',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Referer': 'https://www.mintos.com/lv/statistika/',
}
headers_post = copy.deepcopy(headers_get)
headers_post['Referer'] = 'https://www.ss.lv/lv/transport/cars/volvo/'
headers_post['Content-Type'] = 'application/x-www-form-urlencoded'

session = requests.Session()
response = session.get("https://www.ss.lv/lv/transport/cars/volvo", headers=headers_get)

filters = {
    'topt[8][min]': '',
    'topt[8][max]': '',
    'topt[18][min]': 2015,
    'topt[18][max]': '',
    'topt[15][min]': '',
    'topt[15][max]': '',
    'opt[34]': '',
    'opt[35]': 497,
    'opt[32]': '',
    'opt[17]': '',
    'sid': '/lv/transport/cars/volvo/filter/'
}
response = session.post("https://www.ss.lv/lv/transport/cars/volvo/filter/", headers=headers_post, data=filters)

# print(response.request.body)
# print(response.request.headers)

response = session.get("https://www.ss.lv/lv/transport/cars/volvo/filter/", headers=headers_get)

# print(session.cookies.get_dict())

with open('ss_page1.html', 'w', encoding='utf-8') as file:
    file.write(response.text)

page_nr = 2
while(True):
    response = session.get("https://www.ss.lv/lv/transport/cars/volvo/filter/page{}.html".format(page_nr), headers=headers_get)
    with open('ss_page{}.html'.format(page_nr), 'w', encoding='utf-8') as file:
        file.write(response.text)
    page_nr += 1
    #FIXME: Have to implement end-of-pages check
    if page_nr > 5:
        break


In [49]:
for page_nr in range(1,5):    
    page = open('ss_page{}.html'.format(page_nr), encoding='utf-8')
    soup = BeautifulSoup(page.read())

    table = soup.find('table', {'align': 'center'})

    rows = table.find_all('tr')
    for row in rows[1:]:
        row_data = []
        for cell in row.find_all('td'):
            if len(row) != 8: 
                break
            cell_data = cell.text.strip().replace("\n","")
            if cell_data != '':
                row_data.append(cell.text.strip().replace("\n",""))
        if len(row_data) > 0:
            print(row_data)

["Volvo V60 D4 140kW R-Design komplektācija. Auto atvests no Šveices'M", 'V60', '2016', '2.0D', '213 tūkst.', '14,500  €']
['Volvo V40 2, 0D.  Ādas salons.  NavigācijaAutomašīnas vēsture ir pār', 'V40', '2015', '2.0D', '151 tūkst.', '10,990  €']
['Volvo V60 Momentum A/t. 2015. gada. 2, 0l dīzelis, 88kw, Garantija.  - T', 'V60', '2015', '2.0D', '185 tūkst.', '12,900  €']
['Volvo v60 cross country /automāts /2.0d /110kwPirmā reģ. 20.04.2016A', 'V60', '2016', '2.0D', '226 tūkst.', '16,700  €']
['Uzņēmums pārdod darba auto. Auto pirkts Mūsa motors Latvijā. Apskates veik', 'XC 90', '2018', '2.0D', '83 tūkst.', '37,000  €']
['Volvo Xc60, atpakaļ saktu kamera, 140kw Awd, 4x4, dīzelis, Automātiskā kārba', 'XC 60', '2017', '2.4D', '-', '21,900  €']
['Volvo Xc60 4X4, summum komplektācija, 6 pakāpju automātiskā kārba, 162 KW, 2', 'XC 60', '2015', '2.4D', '-', '19,800  €']
['Xenium Summum, 2.4D, Magic Blue, Awd pilnpiedziņa, vispilnākā versija, bez n', 'XC 60', '2016', '2.4D', '92 tūkst.', '23,900